In [39]:
import mordred
!pip install rdkit
!pip install mordred
!pip install pubchempy


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [40]:
import pandas as pd
import numpy as np
import pubchempy as pcp
from mordred import Calculator, descriptors
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors
from rdkit.Chem import rdMolDescriptors
from urllib.request import urlopen
from urllib.parse import quote

For homework, we chose 8 datasets

In [41]:
df = pd.read_csv(r"C:\Users\user\Desktop\python\8.csv")

Data Preparation
For now, we remove all lines where there is nothing in Name

In [42]:
df = df[~df['Name'].isna()]
df = df.drop_duplicates().reset_index(drop=True)
df

,DOI,Date,Journal,Title,Name,measurement_error,measurement_wavelength,measurement_method,normalised_name,raw_value,specifier
0,10.1039/C6RA04584G_© The Royal Society of Chem...,NaN,NaN,NaN,silicate,0.04,NaN,snowball,NaN,1.21 – 1.13,refractive index
1,10.1016/j.mssp.2013.07.031,8/24/2013,Materials Science in Semiconductor Processing,PREPARATIONCHARACTERIZATIONAPPLICATIONRFSPUTTE...,BSG,0.00,NaN,el_mylogic,"[['B', 1.0], ['SG', 1.0]]",1.41,Refractive index (n)
2,10.1039/C7NR03175K,NaN,NaN,NaN,silica,0.00,NaN,snowball,O=[Si]=O,1.48,refraction index
3,10.1016/j.saa.2012.10.018,10/24/2012,Spectrochimica Acta Part A: Molecular and Biom...,SOLVENTEFFECTABSORPTIONFLUORESCENCESPECTRA7ACE...,DMF,0.00,NaN,el_mylogic,CN(C)C=O,1.4305,n
4,10.1016/j.conbuildmat.2018.05.016Construction ...,NaN,NaN,NaN,silica,0.00,NaN,snowball,O=[Si]=O,1.46,refractive index
...,...,...,...,...,...,...,...,...,...,...,...
4975,10.1016/j.poly.2011.03.003,3/17/2011,Polyhedron,PHOTOPHYSICALPROPERTIESPHOTOCYTOTOXICITYNOVELP...,2-Propanol,0.00,NaN,el_mylogic,NaN,1.377,Refractive index (20 °C)
4976,10.1016/j.optlastec.2019.01.028,2/6/2019,Optics & Laser Technology,SPECTROCHEMICALSTUDYMOLDAVITESRIESIMPACTSTRUCT...,MgO,0.00,NaN,el_cde_tables,"[['Mg', 1.0], ['O', 1.0]]",1.75,n
4977,10.1016/j.ssc.2010.03.031,4/8/2010,Solid State Communications,BROADBAND15MEMISSIONHIGHERBIUMDOPEDBI2O3B2O3GA...,Silicate,0.00,NaN,el_cde_tables,NaN,1.585,Refractive index
4978,10.1016/j.molliq.2017.05.096,5/23/2017,Journal of Molecular Liquids,SOLVATOCHROMICANALYSISDFTCOMPUTATIONALSTUDYAZO...,Diethylether,0.00,NaN,el_mylogic,CCOCC,1.3526,n


Replace unnecessary characters with commas to split the string

In [43]:
df['raw_value'] = df['raw_value'].str.replace('–', ',')
df['raw_value'] = df['raw_value'].str.replace('±', ',')
df['raw_value'] = df['raw_value'].str.replace('~', ',')
df['raw_value'] = df['raw_value'].str.replace(' , ', ',')
df['raw_value'] = df['raw_value'].str.replace(' - ', ',')
df['raw_value'] = df['raw_value'].str.replace(' to ', ',')
df['raw_value'] = df['raw_value'].str.replace('/', ',')

In [44]:
df = df.join(
    df['raw_value'].str.split(',', expand=True).rename(columns={0:'r_v2', 1:'r_v3'}))

Split the numbers between signs into [r_v2] and [r_v3] columns

In [45]:
import re

# define a regular expression pattern to match numbers
pattern = re.compile(r'\d+\.?\d*')

# apply the pattern to each element in the list and convert to float
float
df['r_v2'] = df['r_v2'].apply(lambda x: float(pattern.search(str(x)).group()) if pattern.search(str(x)) else None)
df['r_v3'] = df['r_v3'].apply(lambda x: float(pattern.search(str(x)).group()) if pattern.search(str(x)) else None)

In [46]:
df[['r_v2', 'r_v3']]

,r_v2,r_v3
0,1.2100,1.13
1,1.4100,NaN
2,1.4800,NaN
3,1.4305,NaN
4,1.4600,NaN
...,...,...
4975,1.3770,NaN
4976,1.7500,NaN
4977,1.5850,NaN
4978,1.3526,NaN


Found the mean and put the values in the column [raw_value_clean]

In [47]:
df[pd.to_numeric(df['r_v2'], errors='coerce').notna()]
df[pd.to_numeric(df['r_v3'], errors='coerce').notna()]
df['raw_value_clean'] = df[['r_v2', 'r_v3']].mean(skipna=True, axis=1)
df[['raw_value', 'raw_value_clean']]

,raw_value,raw_value_clean
0,"1.21 , 1.13",1.1700
1,1.41,1.4100
2,1.48,1.4800
3,1.4305,1.4305
4,1.46,1.4600
...,...,...
4975,1.377,1.3770
4976,1.75,1.7500
4977,1.585,1.5850
4978,1.3526,1.3526


Display values less than 1 and replace them with NaN and delete empty values withe whole row

In [48]:
less_than_one = df.loc[df['raw_value_clean'] < 1, 'raw_value_clean']
df.loc[df['raw_value_clean'] < 1, 'raw_value_clean'] = pd.np.nan
df = df.dropna(subset=['raw_value_clean'])

C:\Users\user\AppData\Local\Temp\ipykernel_10988\2985116686.py:2: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.
  df.loc[df['raw_value_clean'] < 1, 'raw_value_clean'] = pd.np.nan


Identify outliers and replace them with NaN, then delete them

In [49]:
Q1 = df['raw_value_clean'].quantile(0.25)
Q3 = df['raw_value_clean'].quantile(0.75)
IQR = Q3 - Q1
df.loc[(df['raw_value_clean'] < Q1 - 1.5 * IQR) | (df['raw_value_clean'] > Q3 + 1.5 * IQR), 'raw_value_clean'] = pd.np.nan
df = df.dropna(subset=['raw_value_clean'])

C:\Users\user\AppData\Local\Temp\ipykernel_10988\4137595389.py:4: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.
  df.loc[(df['raw_value_clean'] < Q1 - 1.5 * IQR) | (df['raw_value_clean'] > Q3 + 1.5 * IQR), 'raw_value_clean'] = pd.np.nan
